In [1]:
import sympy as sym
from os import path
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [2]:
DoFs = 1
# heave and fold angle as degrres of freedom
p = ma.DynamicModelParameters(DoFs)
p.DoFs  = ma.ModelSymbol(value = DoFs,string = 'DoFs')
#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g', real=True)                     # gravity
p.g_v = ma.ModelMatrix(value =[0,0,-1],length=3,string='g_v',real=True)         # gravity vector
p.alpha = ma.ModelSymbol(value = 0,string = 'alpha', real=True)                 # AoA
p.beta = ma.ModelSymbol(value = 0,string = 'beta', real=True)                   # SideSlip
p.v_c = ma.ModelMatrix(value =[0,0,0],length=3,string='v_c', real=True)         # CoM of Wingtip
p.v_lc = ma.ModelMatrix(value =[0,0,0],length=3,string='v_lc', real=True)       # CoM of Liquid
p.v_h = ma.ModelMatrix(value =[0,0,0],length=3,string='v_h', real=True)         # Location of hinge in wing frame
p.m = ma.ModelSymbol(value = 0,string = 'm', real=True)                         # mass of wingtip
p.m_i = ma.ModelSymbol(value = 0,string = 'm_i', real=True)                     # mass of inner wing
p.k_i = ma.ModelSymbol(value = 0,string = 'k_i', real=True)                     # stiffness of inner wing spring
p.m_l = ma.ModelSymbol(value = 0,string = 'm_l', real=True)                     # mass of liquid
p.I_xx = ma.ModelSymbol(value = 0,string = 'I_xx', real=True)                   # wingtip I_xx
p.I_yy = ma.ModelSymbol(value = 0,string = 'I_yy', real=True)                   # wingtip I_yy
p.I_zz = ma.ModelSymbol(value = 0,string = 'I_zz', real=True)                   # wingtip I_zz
p.Lambda = ma.ModelSymbol(value = 0,string = 'Lambda', real=True)               # flare angle

y_0,x_0,z_0 = sym.symbols('y_0,x_0,z_0')
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')

In [3]:
body_frame = ma.frames.HomogenousFrame().Translate(0,0,p.q[0]).R_z(p.beta).R_y(p.alpha).simplify()
hinge_frame = body_frame.Translate(*p.v_h).simplify()
fwt_frame = hinge_frame.simplify()
# body_frame = ma.frames.HomogenousFrame().Translate(0,0,p.q[0]).simplify()
# hinge_frame = body_frame.Translate(*p.v_h).R_x(-p.q[1]).simplify()
# fwt_frame = hinge_frame.simplify()

In [4]:
#Create Elemnts
M = ma.elements.MassMatrix(p.m,p.I_xx,p.I_yy,0,0,0,0)
M_l = ma.elements.MassMatrix(p.m_l,0,0,0,0,0,0)
## FWT mass element
fwt_ele = ma.elements.RigidElement(p.q, fwt_frame, M, grav_vec=p.g_v*p.g, com_pos=p.v_c, name='fwt',simplify = False)
## liquid mass element
liquid_ele = ma.elements.RigidElement(p.q, fwt_frame, M_l, grav_vec=p.g_v*p.g, com_pos=p.v_lc, name='liquid',simplify = False)

spring_ele = ma.elements.Spring(p.q, p.q[0], p.k_i,'Inner Spring')

In [5]:
sm = ma.SymbolicModel.FromElementsAndForces(p.q,[fwt_ele,liquid_ele,spring_ele])

Generating EoM for Element 1 out of 3 - fwt:RigidElement
Generating EoM for Element 2 out of 3 - liquid:RigidElement
Generating EoM for Element 3 out of 3 - Inner Spring:Spring


In [6]:
funcs = []
X = sym.MatrixSymbol('X',3,1)
funcs.append(('get_fwt_V_b',fwt_ele.frame.A.T*fwt_ele.frame.transform_point(X).diff(t)))
funcs.append(('get_fwt_V_global',fwt_ele.frame.transform_point(X).diff(t)))
funcs.append(('get_pos_global2fwt',fwt_ele.frame.transform_global_point(X)))
funcs.append(('get_pos_fwt2global',fwt_ele.frame.transform_point(X)))
funcs.append(('get_fwt_V_b_body',body_frame.A.T*fwt_ele.frame.transform_point(X).diff(t)))
funcs.append(('get_fwt_A',fwt_ele.frame.A))
funcs.append(('get_body_A',body_frame.A))

In [7]:
#create function for an arbitary force of FWT
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W_n = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
W = sym.MatrixSymbol('W',6,1)
fwt_Q_tmp = fwt_ele.frame.Translate(*X).BodyJacobian(p.q).T*W_n
fwt_Q_tmp = me.msubs(fwt_Q_tmp,{**{W_n[i]:W[i,0] for i in range(6)}})
idx = [i for i, e in enumerate(p.q) if e in fwt_ele.q]
fwt_Q = sym.zeros(len(p.q),1)
for i,i_fwt in zip(idx,fwt_ele._idx):
    fwt_Q[i] += fwt_Q_tmp[i_fwt]
funcs.append(('get_fwt_Q',fwt_Q))

In [8]:
#create function for a moment about the hinge
M_h = sym.symbols('M_h')
fwt_Q = sym.zeros(len(p.q),1)
fwt_Q[0] = M_h
funcs.append(('get_hinge_Q',fwt_Q))

In [9]:
funcs.append(('get_fold',p.q[0]))
funcs.append(('get_fold_dt',p.qd[0]))

In [10]:
# create function for acceleration of the fuel tank 
acc = fwt_ele.frame.transform_point(X).diff(t,2) + p.g_v*p.g
A = sym.MatrixSymbol('A',p.DoFs,1)
acc = me.msubs(acc,{p.q[i].diff(t,2):A[i,0] for i in range(DoFs)})
funcs.append(('get_tank_accel',acc))

In [11]:
base_dir = "C:\\Git\\ForJDC_cxx\src_cxx"
class_name = "OneDof_spring"
base_class = "BaseRC"
sm.to_cxx_class(p,base_dir,class_name,base_class,additional_funcs=funcs,additional_includes=['"../BaseRC/BaseRC.hpp"'])
base_class = "mbd.BaseRC"
base_dir = "C:\\Git\\ForJDC_cxx\src_matlab"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable X found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable X found in function get_pos_fwt2global. It will be added to the function signature.
Unknown variable W found in function get_fwt_Q. It will be added to the function signature.
Unknown variable M_h found in function get_hinge_Q. It will be added to the function signature.
Unknown variable A found in function get_tank_accel. It will be added to the function signature.
Unknown variable X found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable X found in function get_pos_fwt2global. It will be added to the function signature.
Unknown variable W found in function get_fwt_Q. It will be added to the function signature.
Unknown variable M_h found in function get_hinge_Q. It will be added to the function signature.
Unknown variable A found in function get_tank_accel. It will be added to the function signature.


In [12]:
# sm.to_cxx_class(p,base_dir,class_name,base_class,additional_funcs=funcs)
# sm.to_cxx_class(p,base_dir,class_name,base_class)